### CODE TO GENERATE PW.X-QUANTUM ESPRESSO INPUT FILE TEMPLATES FROM VARIOUS FORMAT OPTIONS ###

This code is specially useful to obtain the information required by quantum espresso to specify the structure of the material and/or system, e.g. Atomic Positions (angstrom), Cell Parameters (angstrom), ibrav, nat, ntyp.

Currently admited formats:
- .cif (CIF file)
- .vasp (POSCAR/CONTCAR)
- .sdf (SDF file)
- .xyz or extended xyz
- .xsf (XCrySden Structure File)
- .cube (CUBE file)
- .pdb (Protein Data Bank)
- .xtd (Materials Studio file)

This code can be edited accordingly to add more format options if they are available at https://wiki.fysik.dtu.dk/ase/ase/io/io.html

There is an issue when trying to convert .xsf files

In [83]:
import ase
from ase.build import bulk
from ase.io import read,write,cif,vasp,sdf,xyz,extxyz,xsf,xtd,cube,espresso,proteindatabank
from ase.io.espresso import write_espresso_in
from ase import Atoms
from ase.visualize import view

In [84]:
# INFORMATION REQUIRED TO READ THE FILE CONTAINING THE STRUCTURE AND CREATE THE NEW QUANTUM ESPRESSO INPUT FILE #

structure_file = '/home/agarcia/Arturo_MATS/Hexaborides_project/CaB6/supercells/1x1_slab/6_vacuum/CaB6_1x1_6vacuum.vasp' # Path to the file which contains the structure in any of the allowed formats.
qe_input_file_name = '/home/agarcia/Arturo_MATS/Scripts/TEST/TEST_2.in' # Desired path and name for the quantum espresso input file.

In [85]:
# EDIT AND/OR ADD INFORMATION BELOW TO PROPERLY SET THE QUANTUM ESPRESSO INPUT FILE PARAMETERS (OPTIONAL) #
# More documentation on this: 

input_data = {
    'calculation': 'relax',
    'restart_mode': 'from_scratch',
    'tprnfor': True,
    'etot_conv_thr': 1e-5,
    'forc_conv_thr': 1e-4,
    'ecutwfc': 60,
    'ecutrho': 480,
    'input_dft': 'rpbe',
    'vdw_corr': 'dft-d3',
    'occupations': 'smearing',
    'degauss': 0.01,
    'smearing': 'cold',
    'conv_thr': 1e-8,
    'mixing_beta': 0.7,
    'ion_dynamics': 'bfgs',
}  # This flat dictionary will be converted to a nested dictionary where, for example, "calculation" will be put into the "control" section

# If you want to set the exact pseudopotentials please modify the pseudo_body variable in the set_pseudopotentials function.

#### Program Functions ####

In [86]:
# PROGRAM FUNCTIONS #

def get_format(input_file):
    """
    This function gets the format of the file containing the structure and returns a numerical code
    which indicates the specific format. 
    """
    
    format = ''
    
    # Check if the input file is in cif format and return 1 if it is.
    if '.cif' in input_file:
        format = 1
    # Check if the input file is in vasp format and return 2 if it is.
    elif '.vasp' in input_file:
        format = 2
    # Check if the input file is in sdf format and return 3 if it is.
    elif '.sdf' in input_file:
        format = 3
    # Check if the input file is in xyz format and return 4 if it is.
    elif '.xyz' in input_file:
        format = 4
    # Check if the input file is in xsf format and return 5 if it is.
    elif '.xsf' in input_file:
        format = 5
    # Check if the input file is in cube format and return 6 if it is.
    elif '.cube' in input_file:
        format = 6
    # Check if the input file is in pdb format and return 7 if it is.
    elif '.pdb' in input_file:
        format = 7
    # Check if the input file is in xtd format and return 8 if it is.
    elif '.xtd' in input_file:
        format = 8
    else: 
        print('The file format is not admited')
        format = 0

    return format

def get_atoms(format_code, input_file):
    """
    This function asks for the input file containing the structure and the numerical code which indicates the format of the file 
    and calls the appropiate ase.read function to obtain the structure as an atoms object.
    """

    # cif format
    if format_code == 1:
       atoms = ase.io.cif.read_cif(input_file)
        
    # vasp format    
    elif format_code == 2:
        atoms = ase.io.vasp.read_vasp(input_file)

    # sdf format
    elif format_code == 3:
        atoms = ase.io.sdf.read_sdf(input_file)

    # xyz or extxyz format
    elif format_code == 4:
        try:
            atoms = ase.io.extxyz.read_extxyz(input_file)
        except:
            atoms = ase.io.xyz.read_xyz(input_file)

    # xsf format
    elif format_code == 5:
        atoms = ase.io.xsf.read_xsf(input_file)

    # cube format
    elif format_code == 6:
        atoms = ase.io.cube.read_cube(input_file)

    # pdb format
    elif format_code == 7:
        atoms = ase.io.proteindatabank.read_proteindatabank(input_file)

    # xtd format
    elif format_code == 8:
        atoms = ase.io.xtd.read_xtd(input_file)

    return atoms

def set_pseudopotentials(atoms):
    """
    This function gets the atomic species that are present in the system from the atoms object, and uses this information
    to set the pseudopotentials templates (or reals if desired).
    """

    chemical_symbols = atoms.get_chemical_symbols() # Get a list with the atomic symbols of all elements present in the system

    # Remove repeated atomic symbols
    seen = []
    atomic_species = []
    
    for symbol in chemical_symbols:
        if symbol not in seen:
            seen.append(symbol)
            atomic_species.append(symbol)

    # Create the arguments of the pseudopotentials object
    pseudopotentials = {}
    pseudo_body = '.example.pbe-rrkjus.UPF'
    
    for atom in atomic_species:
        pseudo_template = atom + pseudo_body
        pseudopotentials[atom] = pseudo_template

    return pseudopotentials
    

### MAIN PROGRAM ###

In [87]:
# MAIN PROGRAM #

format_code = get_format(structure_file)
structure = get_atoms(format_code, structure_file)
pseudopotentials = set_pseudopotentials(structure)

# Using the atomic coordinates (structure) from the atoms object to create a Quantum ESPRESSO input file template using the pseudopotentials as well as the parameters specified in the first section of the notebook.
write(qe_input_file_name, structure, input_data=input_data, pseudopotentials=pseudopotentials, format='espresso-in')